# Hardware Compatible Gradient Computation on Quantum Hardware

This notebook explores the performance of different gradient computation techniques for quantum hardware.

In [1]:
import pennylane as qml
from pennylane import numpy as np

from context import qnetvo as QNopt

In [2]:
from qiskit import IBMQ

# For details regarding integration between PennyLane and IMB Q,
# see https://pennylaneqiskit.readthedocs.io/en/latest/devices/ibmq.html#accounts-and-tokens
provider = IBMQ.load_account()

## Setup

We prepare a static bell state and optimize the CHSH violation over measurements in the $xz$-plane. 

In [3]:
prep_nodes = [
    QNopt.PrepareNode(1, [0,1], QNopt.ghz_state, 0)
]
meas_nodes = [
    QNopt.MeasureNode(2, 2, [0], QNopt.local_RY, 1),
    QNopt.MeasureNode(2, 2, [1], QNopt.local_RY, 1)
]

We use the `defualt.qubit` device to evaluate the cost function.

In [4]:
local_chsh_ansatz = QNopt.NetworkAnsatz(prep_nodes, meas_nodes)
chsh_cost = QNopt.chsh_inequality_cost(local_chsh_ansatz)

We test the gradients computations on the IBM hwardware simulator and the `ibmq_belem` device.

In [5]:
dev_ibm_qasm = {
    "name" : "qiskit.ibmq",
    "shots" : 4000,
    "backend" : "ibmq_qasm_simulator",
    "provider" : provider
}
dev_ibm_belem = {
    "name" : "qiskit.ibmq",
    "shots" : 4000,
    "backend" : "ibmq_belem",
    "provider" : provider
}

ibm_sim_chsh_ansatz = QNopt.NetworkAnsatz(
    prep_nodes, meas_nodes, dev_kwargs = dev_ibm_qasm
)
belem_chsh_ansatz = QNopt.NetworkAnsatz(
    prep_nodes, meas_nodes, dev_kwargs = dev_ibm_belem
)

## Simulator Runs

We first demonstrate three different gradients on the IBM hardware simulator including parameter shift rule, finite differences rule, and the quantum natural gradient.
In each of these cases, the gradients are computed using 4 parallelized threads.

### Parameter Shift Rule

In [6]:
%%time

param_shift_grad = QNopt.parallel_chsh_grad(
    ibm_sim_chsh_ansatz, diff_method="parameter-shift"
)

np.random.seed(73)
rand_settings = local_chsh_ansatz.rand_scenario_settings()

param_shift_opt = QNopt.gradient_descent(
    chsh_cost,
    rand_settings,
    num_steps=1,
    step_size=0.1,
    sample_width=1,
    grad_fn=param_shift_grad
)

print("max score : ", param_shift_opt["opt_score"])

iteration :  0 , score :  1.7457550793577905
max score :  1.9621092378382046
CPU times: user 1.82 s, sys: 162 ms, total: 1.99 s
Wall time: 34.2 s


### Finite Differences Rule

In [20]:
%%time

finite_diff_grad = QNopt.parallel_chsh_grad(ibm_sim_chsh_ansatz, diff_method="finite-diff")

np.random.seed(73)
rand_settings = local_chsh_ansatz.rand_scenario_settings()

finite_diff_opt = QNopt.gradient_descent(
    chsh_cost,
    rand_settings,
    num_steps=1,
    step_size=0.0000001,
    sample_width=1,
    grad_fn=finite_diff_grad
)

print("max score : ", finite_diff_opt["opt_score"])

iteration :  0 , score :  1.7457550793577905
max score :  1.7499837568803895
CPU times: user 1.33 s, sys: 116 ms, total: 1.44 s
Wall time: 32.9 s


### Quantum Natural Gradient

This approach requires 4 extra calls to the quantum hardware resulting in an increased overhead, however, it may show some optimization advantages over other approaches in practice on noisy hardware.

In [8]:
%%time

natural_grad = QNopt.chsh_natural_grad(
    ibm_sim_chsh_ansatz, diff_method="parameter-shift"
)

np.random.seed(73)
rand_settings = local_chsh_ansatz.rand_scenario_settings()

natural_grad_opt = QNopt.gradient_descent(
    chsh_cost,
    rand_settings,
    num_steps=1,
    step_size=0.1,
    sample_width=1,
    grad_fn=natural_grad
)

print("max score : ", natural_grad_opt["opt_score"])

iteration :  0 , score :  1.7457550793577905
max score :  2.1185506222910355
CPU times: user 1.97 s, sys: 183 ms, total: 2.15 s
Wall time: 52.2 s


## Quantum Hardware Runs

### Parameter Shift Rule

In [9]:
%%time

param_shift_grad = QNopt.parallel_chsh_grad(
    belem_chsh_ansatz, diff_method="parameter-shift"
)

np.random.seed(73)
rand_settings = local_chsh_ansatz.rand_scenario_settings()

belem_param_shift_opt = QNopt.gradient_descent(
    chsh_cost,
    rand_settings,
    num_steps=1,
    step_size=0.1,
    sample_width=1,
    grad_fn=param_shift_grad
)

print("max score : ", belem_param_shift_opt["opt_score"])

iteration :  0 , score :  1.7457550793577905
max score :  1.9289879321908119
CPU times: user 1.82 s, sys: 166 ms, total: 1.98 s
Wall time: 3min 21s


### Finite Differences Rule

In [21]:
%%time

finite_diff_grad = QNopt.parallel_chsh_grad(
    belem_chsh_ansatz, diff_method="finite-diff"
)

np.random.seed(73)
rand_settings = local_chsh_ansatz.rand_scenario_settings()

belem_finite_diff_opt = QNopt.gradient_descent(
    chsh_cost,
    rand_settings,
    num_steps=1,
    step_size=0.0000001,
    sample_width=1,
    grad_fn=finite_diff_grad
)

print("max score : ", belem_finite_diff_opt["opt_score"])

iteration :  0 , score :  1.7457550793577905
max score :  1.6986465726018762
CPU times: user 1.44 s, sys: 127 ms, total: 1.57 s
Wall time: 2min 37s


### Quantum Natural Gradient

In [12]:
%%time

natural_grad = QNopt.chsh_natural_grad(
    belem_chsh_ansatz, diff_method="parameter-shift"
)

np.random.seed(73)
rand_settings = local_chsh_ansatz.rand_scenario_settings()

belem_natural_grad_opt = QNopt.gradient_descent(
    chsh_cost,
    rand_settings,
    num_steps=1,
    step_size=0.1,
    sample_width=1,
    grad_fn=natural_grad
)

print("max score : ", belem_natural_grad_opt["opt_score"])

iteration :  0 , score :  1.7457550793577905
max score :  2.10422445187388
CPU times: user 2.13 s, sys: 182 ms, total: 2.32 s
Wall time: 4min 9s


## Conclusions

The quantum natural gradient takes longer than the other two options because it requires 8 additional web requests to compute the metric tensor across the four parallel differentiated qnodes.
The natural gradient however appears to perform significantly better on quantum hardware than parameter shift and finite differences.

The finite differences rule may not be optimized in terms of its step size because it does not compute gradients for the CHSH cost function very effectively.